In [ ]:
pip install requests pandas

In [8]:
import requests
import pandas as pd
import time
from datetime import datetime
from google.colab import files


def crawl_tiki_product_ids(category_url, target_reviews=3000):
    """Crawl product IDs và tên sản phẩm từ danh mục Tiki, chỉ lấy sản phẩm có review, cho đến khi đủ số review."""
    product_list = []  # list of {'id': id, 'name': name}
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    all_reviews = []
    page = 1

    while len(all_reviews) < target_reviews:
        url = f"{category_url}&page={page}"
        try:
            resp = requests.get(url, headers=headers)
            if resp.status_code != 200:
                print(f"⚠️ Lỗi khi truy cập trang {page}, mã lỗi: {resp.status_code}")
                break

            data = resp.json().get("data", [])
            if not data:
                print(f"⚠️ Hết sản phẩm tại trang {page} của danh mục")
                break

            new_products = []
            for prod in data:
                pid = prod.get("id")
                name = prod.get("name")
                reviews_count = prod.get("review_count", 0)
                if pid and reviews_count > 0:
                    new_products.append({"id": pid, "name": name})

            print(f"Trang {page} | Số sản phẩm có review: {len(new_products)}")

            for prod in new_products:
                print(f"\n📦 Crawl reviews cho sản phẩm ID: {prod['id']} - {prod['name']}")
                reviews = get_tiki_reviews(prod['id'], prod['name'], max_reviews=50)
                all_reviews.extend(reviews)
                print(f"🌟 Tổng số review hiện tại: {len(all_reviews)}")
                if len(all_reviews) >= target_reviews:
                    break

            product_list.extend(new_products)
            page += 1
            time.sleep(1)

            if len(all_reviews) >= target_reviews:
                print(f"✅ Đã thu thập đủ {len(all_reviews)} review")
                break

        except Exception as e:
            print(f"❌ Lỗi khi crawl trang {page}: {str(e)}")
            break

    return product_list, all_reviews


def get_tiki_reviews(product_id, product_name, max_reviews=50):
    """Lấy review của một sản phẩm, chỉ giữ review có comment hoặc ảnh."""
    reviews = []
    page = 1
    headers = {"User-Agent": "Mozilla/5.0"}

    while len(reviews) < max_reviews:
        url = f"https://tiki.vn/api/v2/reviews?product_id={product_id}&page={page}&limit=20&sort=score|desc"
        try:
            resp = requests.get(url, headers=headers)
            if resp.status_code != 200:
                print(f"⚠️ Lỗi trang {page} sản phẩm {product_id}, mã lỗi: {resp.status_code}")
                break

            items = resp.json().get("data", [])
            if not items:
                print(f"Sản phẩm {product_id} | Hết review tại trang {page}")
                break

            for item in items:
                if len(reviews) >= max_reviews:
                    break
                content = item.get("content") or ""
                images = [img.get("full_path") for img in item.get("images", [])]
                # Chỉ lấy nếu có comment hoặc hình ảnh
                if not content.strip() and not images:
                    continue

                # Định dạng ngày
                ts = item.get("created_at")
                try:
                    dt = datetime.fromtimestamp(ts) if ts else None
                    created = dt.strftime("%d/%m/%y") if dt else None
                except:
                    created = None

                reviews.append({
                    "user_name": item.get("created_by", {}).get("name"),
                    "rating": item.get("rating"),
                    "comment": content,
                    "images": images,
                    "created_at": created,
                    "thank_count": item.get("thank_count"),
                    "product_id": product_id,
                    "product_name": product_name
                })
            page += 1
            time.sleep(1)
        except Exception as e:
            print(f"❌ Lỗi crawl review sản phẩm {product_id} trang {page}: {str(e)}")
            break

    print(f"Sản phẩm {product_id} | Thu được {len(reviews)} review có comment hoặc hình ảnh")
    return reviews


def main():
    category_input = input("Nhập danh mục (ID hoặc từ khóa, ví dụ '1815' hoặc 'coffee'): ").strip()
    target_reviews = 3000

    if category_input.isdigit():
        category_url = f"https://tiki.vn/api/v2/products?category={category_input}&limit=48"
    else:
        category_url = f"https://tiki.vn/api/v2/products?q={category_input}&limit=48"

    print(f"🔍 Sử dụng URL: {category_url}")
    print("🔍 Bắt đầu crawl...")

    products, all_reviews = crawl_tiki_product_ids(category_url, target_reviews=target_reviews)
    print(f"✅ Số sản phẩm đã lấy thông tin: {len(products)}")
    print(f"✅ Tổng review thu thập được: {len(all_reviews)}")

    if all_reviews:
        df = pd.DataFrame(all_reviews)
        csv_path = f"tiki_reviews_{time.strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        print(f"✅ Lưu file CSV: {csv_path}")
        files.download(csv_path)
    else:
        print("⚠️ Không có review nào được thu thập.")

if __name__ == "__main__":
    main()


Nhập danh mục (ID hoặc từ khóa, ví dụ '1815' hoặc 'coffee'): 1975
🔍 Sử dụng URL: https://tiki.vn/api/v2/products?category=1975&limit=48
🔍 Bắt đầu crawl...
Trang 1 | Số sản phẩm có review: 34

📦 Crawl reviews cho sản phẩm ID: 277396880 - Quấn cổ tay Có Móc( 1 ĐÔI ) - Tập kéo xà, tập Tạ - TP-7684 - Nâng Tạ,Tập Xà Đơn
Sản phẩm 277396880 | Hết review tại trang 2
Sản phẩm 277396880 | Thu được 0 review có comment hoặc hình ảnh
🌟 Tổng số review hiện tại: 0

📦 Crawl reviews cho sản phẩm ID: 275955767 - BG- Ghế ngoài trời S12 có túi đeo,có thể gập lại,Ghế cắm trại,ghế di động,ghế bãi biển,ghế dã ngoại - Size S
Sản phẩm 275955767 | Hết review tại trang 2
Sản phẩm 275955767 | Thu được 0 review có comment hoặc hình ảnh
🌟 Tổng số review hiện tại: 0

📦 Crawl reviews cho sản phẩm ID: 275716418 - Bình Xăng Thơm Zippo Loại Xịn Chính Hãng 125ml Dùng Cho Hộp Quẹt Zippo, Bật Lửa Zippo Chuyên Dụng
Sản phẩm 275716418 | Hết review tại trang 2
Sản phẩm 275716418 | Thu được 2 review có comment hoặc hình ảnh
🌟 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>